In [1]:
# Third party imports
from IPython.display import display, HTML
from bokeh.plotting import figure, output_file, ColumnDataSource
from bokeh.io import output_notebook, show
from bokeh.models import HoverTool
import numpy as np
from random import randint
# local imports
from libcomcat.search import get_event_by_id
from libcomcat.dataframes import get_detail_data_frame, get_history_data_frame

PRODUCTS = ['dyfi', 'origin']
output_notebook()

Loading BokehJS ...

In [2]:
df = get_history_data_frame('ci38457511', PRODUCTS)
df = df[0]

In [3]:
display(HTML(df.to_html()))

,Update Time,Product,Authoritative Event ID,Code,Associated,Product Source,Product Version,Elapsed (min),Description
1418,2019-07-06 03:22:22.060,origin,ci38457511,ci38457511,True,ci,1,2.5,"Magnitude# 7.1|Time# 2019-07-06 03:19:52.630000 |Time Offset (sec)# -0.4|Location# (35.766,-117.605)|Distance from Auth. Origin (km)# 0.6|Azimuth# SW|Depth# -0.9|Magnitude Type# me|Location Method# unknown|Preferred Weight#156"
0,2019-07-06 03:23:03.442,dyfi,ci38457511,ci38457511,True,us,1,3.2,Max MMI# 3.8|NumResponses# 15
1419,2019-07-06 03:24:53.240,origin,ci38457511,ci38457511,True,ci,2,5.0,"Magnitude# 6.9|Time# 2019-07-06 03:19:52.630000 |Time Offset (sec)# -0.4|Location# (35.766,-117.605)|Distance from Auth. Origin (km)# 0.6|Azimuth# SW|Depth# -0.9|Magnitude Type# mw|Location Method# unknown|Preferred Weight#156"
1417,2019-07-06 03:25:49.786,origin,ci38457511,at00pu7alg,True,at,1,5.9,"Magnitude# 7.1|Time# 2019-07-06 03:20:04 |Time Offset (sec)# 11.0|Location# (35.488,-117.923)|Distance from Auth. Origin (km)# 42.8|Azimuth# SW|Depth# 100.0|Magnitude Type# Mi|Location Method# l|Preferred Weight#6"
1,2019-07-06 03:26:31.983,dyfi,ci38457511,ci38457511,True,us,2,6.6,Max MMI# 9.1|NumResponses# 35
1420,2019-07-06 03:27:00.170,origin,ci38457511,ci38457511,True,ci,3,7.1,"Magnitude# 6.9|Time# 2019-07-06 03:19:52.630000 |Time Offset (sec)# -0.4|Location# (35.766,-117.605)|Distance from Auth. Origin (km)# 0.6|Azimuth# SW|Depth# -0.9|Magnitude Type# mw|Location Method# unknown|Preferred Weight#156"
2,2019-07-06 03:33:07.443,dyfi,ci38457511,ci38457511,True,us,3,13.2,Max MMI# 9.1|NumResponses# 68
1427,2019-07-06 03:33:09.715,origin,ci38457511,pt19187000,True,pt,1,13.3,"Magnitude# 7.1|Time# 2019-07-06 03:19:53 |Time Offset (sec)# -0.0|Location# (35.500,-117.900)|Distance from Auth. Origin (km)# 40.4|Azimuth# SW|Depth# 100.0|Magnitude Type# Mi|Location Method# u|Preferred Weight#6"
1428,2019-07-06 03:37:21.040,origin,ci38457511,us70004bn0,True,us,1,17.5,"Magnitude# 7.1|Time# 2019-07-06 03:19:52.630000 |Time Offset (sec)# -0.4|Location# (35.767,-117.605)|Distance from Auth. Origin (km)# 0.6|Azimuth# SW|Depth# -0.9|Magnitude Type# mww|Location Method# unknown|Preferred Weight#6"
3,2019-07-06 03:39:33.071,dyfi,ci38457511,ci38457511,True,us,4,19.7,Max MMI# 9.1|NumResponses# 121


In [19]:
eventid = 'ci38457511'


p = figure(plot_width=500, plot_height=400, tools="pan,box_zoom,reset,save",
           title="Product History: %s" % eventid,
           x_axis_type="log", x_axis_label="Time (minutes elapsed)", 
           y_axis_label="Number of Products")


for prod in PRODUCTS:
    pdf = df[df['Product'] == prod]
    times = pdf['Elapsed (min)'].values
    description = pdf['Description'].values
    num_origins = np.linspace(1, len(times), len(times))
    str_time = np.asarray(pdf['Update Time'].to_list(), dtype='str')
    codes = pdf['Code'].values
    color = '#%06X' % randint(0, 0xFFFFFF)
    source = ColumnDataSource(data=dict(
        x=times,
        y=num_origins,
        desc=description,
        update=str_time,
        code=codes
    ))
    plot = p.circle('x', 'y', size=6, source=source, fill_color=color, line_color=color, legend=prod)
    TOOLTIPS = [
        ("UpdateTime", "@update"),
        ("Code", "@code"),
        ("desc", "@desc")
    ]
    p.add_tools(HoverTool(renderers=[plot], tooltips=TOOLTIPS,mode='vline'))
    

p.legend.location = "top_left"
p.legend.click_policy="mute"
show(p)